In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [4]:
from pyspark.ml.regression import LinearRegression

In [7]:
data = spark.read.csv('gs://hkanjih-spark-udemy/material/Spark_for_Machine_Learning/Ecommerce-Customers.csv',inferSchema=True, header=True)

In [9]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [11]:
for item in data.head(2)[1]:
  print(item)

hduke@hotmail.com
4547 Archer CommonDiazchester, CA 06566-8576
DarkGreen
31.9262720264
11.1094607287
37.2689588683
2.66403418213
392.204933444


In [15]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler


In [16]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [17]:
assembler = VectorAssembler(inputCols=[ 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership'],outputCol='features')

In [18]:
output = assembler.transform(data)

In [19]:
output.head(1)

[Row(Email=u'mstephenson@fernandez.com', Address=u'835 Frank TunnelWrightmouth, MI 82180-9605', Avatar=u'Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [20]:
final_data = output.select('features','Yearly Amount Spent' )

In [21]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [22]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [23]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                353|
|   mean| 501.92632540596367|
| stddev|  79.60881413234792|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [24]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                147|
|   mean|   493.040994972723|
| stddev|  78.51841591725416|
|    min| 308.52774655803336|
|    max|  744.2218671047146|
+-------+-------------------+



In [25]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [26]:
lr_model = lr.fit(train_data)

In [27]:
test_results = lr_model.evaluate(test_data)

In [28]:
test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|  -4.019533017104266|
|  3.5704167196994376|
|  -6.888091078552236|
|  -7.791872678235677|
|  10.836221125032353|
|  18.101737913433965|
|  2.7118800461772707|
|-0.48863805432597474|
|  17.667613057718427|
|  -4.016003831669934|
| -1.9592958443228667|
|  0.8751462887601633|
| -5.0133423131060795|
| -16.958792483789296|
|    6.43249803372936|
|   -9.42275645322951|
| -2.7593052179215647|
|  -1.428246033663413|
|  -9.649549046046502|
|  -17.60570079876294|
+--------------------+
only showing top 20 rows



In [29]:
test_results.rootMeanSquaredError

9.624761241157541

In [30]:
test_results.r2

0.9848713090837975

In [31]:
unlabeled_data = test_data.select('features')

In [32]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[30.8794843441274...|
|[31.0472221394875...|
|[31.0613251567161...|
|[31.1280900496166...|
|[31.1695067987115...|
|[31.3123495994443...|
|[31.3662121671876...|
|[31.3895854806643...|
|[31.6005122003032...|
|[31.6253601348306...|
|[31.7216523605090...|
|[31.7366356860502...|
|[31.7656188210424...|
|[31.8164283341993...|
|[31.8209982016720...|
|[31.8279790554652...|
|[31.8530748017465...|
|[31.8627411090001...|
|[31.8854062999117...|
|[31.9048571310136...|
+--------------------+
only showing top 20 rows



In [34]:
predictions= lr_model.transform(unlabeled_data)

In [35]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.8794843441274...|494.22613300195894|
|[31.0472221394875...|388.92698246932196|
|[31.0613251567161...|494.44354913645384|
|[31.1280900496166...| 565.0445594252903|
|[31.1695067987115...|416.52030967726046|
|[31.3123495994443...|445.48968011450665|
|[31.3662121671876...|427.87700251030765|
|[31.3895854806643...|410.55824911430886|
|[31.6005122003032...| 461.5052384333785|
|[31.6253601348306...|380.35290458859413|
|[31.7216523605090...| 349.7362224761955|
|[31.7366356860502...|496.05829996677176|
|[31.7656188210424...| 501.5674239487132|
|[31.8164283341993...| 518.0812839874457|
|[31.8209982016720...|  418.242782979484|
|[31.8279790554652...|449.42550400017103|
|[31.8530748017465...|462.04442868027354|
|[31.8627411090001...| 557.7263872077101|
|[31.8854062999117...|  399.752822018522|
|[31.9048571310136...| 491.5555582215791|
+--------------------+------------